# RedPanda Example Usage

## 1. Create Database Model

Create a model representing a table in the DB

In [1]:
import sqlalchemy
import sqlalchemy.ext.declarative
import redpanda.mixins

# Declare the SQLAlchemy declarative base
Base = sqlalchemy.ext.declarative.declarative_base()

# Declare our model
class Widget(redpanda.mixins.RedPandaMixin, Base):
    __tablename__ = 'widgets'
    id            = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp     = sqlalchemy.Column(sqlalchemy.DateTime)
    name          = sqlalchemy.Column(sqlalchemy.String)
    kind          = sqlalchemy.Column(sqlalchemy.String)
    units         = sqlalchemy.Column(sqlalchemy.Integer)

    # Class-defined RedPanda read_sql() arguments
    # This allows us to forego passing these into Widget.redpanda()
    __read_sql__ = {
        'index_col'   : ['timestamp'],
        'parse_dates' : ['timestamp'] }

    def __repr__(self):
        return "<Widget id: %s timestamp: '%s' name: '%s' kind: '%s' units: %d>" % \
            (self.id, self.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                self.name, self.kind, self.units)

## 2. Database Engine

Create an in-memory SQLite engine

In [2]:
engine = sqlalchemy.create_engine('sqlite://')

## 3. Populate the Database

Fill the "widgets" table with some data

In [3]:
from datetime import datetime
import random
import random_words

def randdate(maxday=31):
    """ Generate a random datetime. """
    year = 2015
    month = random.randint(0,12) + 1
    day   = random.randint(0,maxday) + 1
    hour  = random.randint(0,24)
    minute = random.randint(0,60)
    try:
        return datetime(year, month, day, hour, minute)
    except ValueError:
        return randdate(maxday-1)

def widgetgen():
    """ Generate a set of widgets. """
    wordgen = random_words.RandomWords()
    kinds   = 'fizzer', 'buzzer', 'bopper'
    for i in range(0,25):
        for kind in kinds:
            name      = wordgen.random_word()
            timestamp = randdate()
            units     = random.randint(0,100)
            yield Widget(timestamp=timestamp, name=name, kind=kind, units=units)

# Set up our database
Base.metadata.create_all(engine)
sessionmaker = sqlalchemy.orm.sessionmaker(bind=engine)
sessiongen   = sqlalchemy.orm.scoped_session(sessionmaker)
session      = sessiongen()
session.add_all(sorted(widgetgen(), key=lambda x: x.timestamp))
session.commit()

## 4. RedPanda Model-to-DataFrame

Use the `Widget.redpanda()` method to create a framable query. Note because `Widget.__read_sql__` is initialized, the index will be set to `timestamp`.

In [4]:
Widget.redpanda().frame(engine).head()

,id,name,kind,units
timestamp,,,,
2015-01-01 07:00:00,1,analog,bopper,7
2015-01-02 21:06:00,2,seamen,bopper,67
2015-01-10 10:02:00,3,tendency,buzzer,11
2015-01-13 07:18:00,4,apparatus,bopper,2
2015-01-14 06:43:00,5,ornament,bopper,93


### Adding constraints

Limit results to November 2015

In [5]:
Widget.redpanda()\
    .filter(Widget.timestamp.between('2015-11-01', '2015-11-30 23:59:59'))\
    .frame(engine)

,id,name,kind,units
timestamp,,,,
2015-11-03 17:48:00,64,fifteen,buzzer,47
2015-11-05 21:31:00,65,bytes,fizzer,95
2015-11-08 19:10:00,66,profession,buzzer,79
2015-11-26 12:59:00,67,habit,buzzer,58
2015-11-27 16:49:00,68,concentrations,fizzer,52
2015-11-29 12:00:00,69,lifeboat,fizzer,47
2015-11-30 10:18:00,70,porters,buzzer,10


### Aggregation

Flatten table into the sum of units across timegroup vs. kind

In [6]:
import pandas
Widget.redpanda().frame(engine)\
    .groupby([pandas.TimeGrouper("M"), "kind"]).units.sum().unstack()

kind,bopper,buzzer,fizzer
timestamp,,,
2015-01-31,169,200,77
2015-02-28,9,NaN,NaN
2015-03-31,250,3,157
2015-04-30,NaN,89,20
2015-05-31,23,152,30
2015-06-30,NaN,197,68
2015-07-31,33,135,357
2015-08-31,109,130,113
2015-09-30,146,145,288


## RedPanda DataFrame-to-Model

Use `Widget.redparse()` to parse a DataFrame into ORM objects

In [7]:
frame = Widget.redpanda().frame(engine)
modelgen = Widget.redparse(frame, parse_index=True)
print modelgen.next()

<Widget id: 1 timestamp: '2015-01-01 07:00:00' name: 'analog' kind: 'bopper' units: 7>
